In [1]:
!pip install --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
userdata.get('groq_api')

In [3]:
import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


### System prompts for different experts

In [19]:
MODEL_CONFIG = {
    "technical": "You are a technical expert. Answer the query. Your response must be rigorous, code-focused, and precise.",
    "billing": "You are a billing expert. Answer the query. Your response must be empathetic, financial-focused, and policy-driven.",
    "general": "You make small talk. Answer the query. Your response must be casual and friendly.",
    "tool": "You have the ability to use an external API appropriate to fetch an asnwer for the user's query. Your response must be concise."
}

### Setup the prompt router
Light model: llama-3.1-8b-instant

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

router_template = ChatPromptTemplate.from_messages([
    ("system", "Out of 4 categories - 'technical', 'billing' (sales policy related query), 'tool' (queries requiring llm to fetch real time data or external information), 'general' (queries requiring small talk response) - you must correctly choose which category the user query belongs to. respond with only that one word - 'technical', 'biliing', 'tool' or 'general' only using lowercase letters."),
    ("human", "{query}")
])

router_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)
parser = StrOutputParser()

def route_prompt(user_input: str) -> str:
    chain = router_template | router_llm | parser
    return chain.invoke({"query": user_input})

### Setup base model for different experts
Stronger model: llama-3.3-70b-versatile

In [8]:
expert_llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7)

def process_request(user_input: str) -> str:
    category = route_prompt(user_input)
    system_prompt = MODEL_CONFIG[category]

    expert_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])

    chain = expert_template | expert_llm | parser
    return chain.invoke({"query": user_input})

In [20]:
queries = [
    "My python script is throwing an IndexError. What is an IndexError?",
    "I was charged twice for my subscription this month. Is there a refund policy?",
    "The weather seems nice today.",
    "What is the price of bitcoin?"
]

for query in queries:
    print("%"*165)
    print(f"Query: {query}")
    print(f"Response: {process_request(query)}\n")

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Query: My python script is throwing an IndexError. What is an IndexError?
Response: IndexError can be a real pain. So, basically, an IndexError in Python happens when you're trying to access an element in a list (or a string, or a tuple) using an index that doesn't exist. Think of it like trying to get to the 10th floor of a building that only has 5 floors - there's just nothing there.

For example, if you have a list `my_list = [1, 2, 3]`, trying to access `my_list[3]` would raise an IndexError because the list only has indices 0, 1, and 2. Make sense? 

To fix it, you'll need to check your code and make sure you're not trying to access anything that's out of range. Maybe you need to adjust your loop or your indexing logic. Do you want to share your code and I can help you take a closer look?

%%%%%%%%%%%%%%%%%%%%%%%%%%%%

* prompt routing is done succesfully
* the technical, billing and general experts respond as expected
* tool use expert generates the most likely text and talks about using an API to fetch bitcoin details. returns the wrong price.

### Mock data fetch

* if the prompt router model categorises the query as `tool use`, then call a function that "fetches" the required data.

In [5]:
def fetch_data() -> str:
    return "Fetching data...\nData fetched: •••••••"

def process_request(user_input: str) -> str:
    category = route_prompt(user_input)
    if category == "tool":
        return fetch_data()

    system_prompt = MODEL_CONFIG[category]

    expert_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])

    chain = expert_template | expert_llm | parser
    return chain.invoke({"query": user_input})

In [6]:
query = "what is the price of bitcoin?"
print(process_request(query))

Fetching data...
Data fetched: •••••••
